In [16]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import xgboost
from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc
import pickle
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir('./predsales') if isfile(join('./predsales', f))]
items = pd.read_csv('./predsales/' + onlyfiles[0])
shops = pd.read_csv('./predsales/'+ onlyfiles[4])
cats = pd.read_csv('./predsales/'+ onlyfiles[1])
train = pd.read_csv('./predsales/' + onlyfiles[2])
# set index to ID to avoid droping it later
test  = pd.read_csv('./predsales/' + onlyfiles[5]).set_index('ID') 


In [18]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [19]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [20]:
cats.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [21]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [22]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]
shops

,shop_id,city_code
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2
5,5,3
6,6,4
7,7,4
8,8,4
9,9,5


In [23]:
cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code','subtype_code']]

items.drop(['item_name'], axis = 1, inplace = True)
cats

,item_category_id,type_code,subtype_code
0,0,0,29
1,1,1,9
2,2,1,10
3,3,1,11
4,4,1,13
5,5,1,14
6,6,1,16
7,7,1,17
8,8,2,26
9,9,3,33


In [24]:
len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len((set(test.item_id))), len(set(train.item_id)),len(test)

(363, 5100, 21807, 214200)

In [25]:
len(set(test.item_id) - (set(train.item_id)))

363

In [26]:
ts = time.time()
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    


In [27]:
matrix = pd.DataFrame(np.vstack(matrix), columns = cols)
matrix[cols[0]] = matrix[cols[0]].astype(np.int8)
matrix[cols[1]] = matrix[cols[1]].astype(np.int8)
matrix[cols[2]] = matrix[cols[2]].astype(np.int16)
matrix.sort_values(cols,inplace=True)

In [29]:
np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16')


array([[   33,    45, 13315],
       [   33,    45, 13880],
       [   33,    45, 13881],
       ...,
       [   33,    21,  7640],
       [   33,    21,  7632],
       [   33,    21,  7440]], dtype=int16)

In [31]:
# take absolute value of daily sales volume and calculate daily revenue
train['item_cnt_day'] = train['item_cnt_day'].map(lambda x: abs(x))
train['revenue'] = train['item_price'] * train['item_cnt_day']

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,revenue
0,02.01.2013,0,59,22154,999.00,1.0,999.00
1,03.01.2013,0,25,2552,899.00,1.0,899.00
2,05.01.2013,0,25,2552,899.00,1.0,899.00
3,06.01.2013,0,25,2554,1709.05,1.0,1709.05
4,15.01.2013,0,25,2555,1099.00,1.0,1099.00
5,10.01.2013,0,25,2564,349.00,1.0,349.00
6,02.01.2013,0,25,2565,549.00,1.0,549.00
7,04.01.2013,0,25,2572,239.00,1.0,239.00
8,11.01.2013,0,25,2572,299.00,1.0,299.00
9,03.01.2013,0,25,2573,299.00,3.0,897.00


In [37]:
grouping 

group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)
group

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
5,0,0,61,1.0
6,0,0,75,1.0
7,0,0,88,1.0
8,0,0,95,1.0
9,0,0,96,1.0


In [34]:
matrix = pd.merge(matrix,group, on = cols, how ='left',)

In [35]:
matrix['item_cnt_month'] = (matrix['item_cnt_month'].fillna(0).clip(0,20).astype(np.float16))

In [36]:
matrix

,date_block_num,shop_id,item_id,item_cnt_month
0,0,0,19,0.0
1,0,0,27,0.0
2,0,0,28,0.0
3,0,0,29,0.0
4,0,0,32,6.0
5,0,0,33,3.0
6,0,0,34,0.0
7,0,0,35,1.0
8,0,0,40,0.0
9,0,0,41,0.0


In [39]:
# adding test data together with training set
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [41]:
matrix = pd.concat([matrix,test],ignore_index = True,sort=False,keys=cols)
matrix.fillna(0,inplace= True)

In [43]:
# left joining shop,item, and category tables
matrix = pd.merge(matrix,shops,on=['shop_id'],how = 'left')
matrix = pd.merge(matrix,items,on = ['item_id'], how = 'left')
matrix = pd.merge(matrix,cats, on = ['item_category_id'], how ='left')

# force data type to save memory
matrix['city_code'] = matrix['city_code'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)

In [50]:
# lag function

def lag_target(df,lag,col):
    temp = df[['date_block_num','shop_id','item_id',col]]
    
    for i in lag:
        shifted = temp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id',col+'_lag_'+str(i)]
        shifted['date_block_num'] += 1 
        df = pd.merge(df,shifted, on =['date_block_num','shop_id','item_id'],how = 'left')
        
    return df

x = lag_target(matrix,[1,2,3,6,12],'item_cnt_month')
x

,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0,0,19,0.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0.0,29,19,5,10,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0.0,29,30,8,55,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0.0,29,23,5,16,NaN,NaN,NaN,NaN,NaN
4,0,0,32,6.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
5,0,0,33,3.0,29,37,11,1,NaN,NaN,NaN,NaN,NaN
6,0,0,34,0.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
7,0,0,35,1.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
8,0,0,40,0.0,29,57,13,8,NaN,NaN,NaN,NaN,NaN
9,0,0,41,0.0,29,57,13,8,NaN,NaN,NaN,NaN,NaN


In [49]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')


,date_block_num,shop_id,item_id,item_cnt_month,city_code,item_category_id,type_code,subtype_code,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0,0,19,0.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
1,0,0,27,0.0,29,19,5,10,NaN,NaN,NaN,NaN,NaN
2,0,0,28,0.0,29,30,8,55,NaN,NaN,NaN,NaN,NaN
3,0,0,29,0.0,29,23,5,16,NaN,NaN,NaN,NaN,NaN
4,0,0,32,6.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
5,0,0,33,3.0,29,37,11,1,NaN,NaN,NaN,NaN,NaN
6,0,0,34,0.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
7,0,0,35,1.0,29,40,11,4,NaN,NaN,NaN,NaN,NaN
8,0,0,40,0.0,29,57,13,8,NaN,NaN,NaN,NaN,NaN
9,0,0,41,0.0,29,57,13,8,NaN,NaN,NaN,NaN,NaN
